<img src=".\imagens_rastros\36393436-set-von-fußabdrücken-schuhe-symbol-horizontale-spuren-mann.jpg" alt="Drawing" style="width: 300px;"/>

## O Jogo dos Rastros

Este jogo é da autoria de Bill Taylor (1992). Pertence a uma família de jogos onde cada casa por onde as peças movimentadas passam deixa de estar disponível. Desta forma, o número de possibilidades diminui rapidamente garantindo partidas curtas.

### Material
Um tabuleiro quadrado (tipicamente 8 por 8), uma peça branca e pedras negras suficientes. Inicialmente, existe apenas uma peça no tabuleiro, a peça branca, que começa sempre no quadrado 4-5. Uma casa é identificada pelo par linha-coluna.


<img src=".\imagens_rastros\tab1.png" alt="Drawing" style="width: 250px;"/>

### Objectivo do jogo
Este jogo é jogado por dois jogadores, chamemos ao primeiro o jogador sul, que abre o jogo, e ao seu adversário o jogador norte. Cada jogador deseja colocar a peça branca num quadrado específico, o jogador sul quer atingir o quadrado etiquetado como s (8-1) que fica no fundo à esquerda (por isso lhe chamamos sul) e o segundo jogador quer atingir o quadrado n (1-8) que fica na zona norte do tabuleiro. Ganha o jogador que consiga levar primeiro a peça branca à sua casa final. Mesmo que seja o adversário a deslocar a peça para a casa final de um dos jogadores, este ganha. Vamos também adoptar uma variante do jogo original que define que um jogador ganha também se colocar a peça branca numa casa onde o adversário não possa jogar, bloqueando-o. 

### Regras
Os jogadores vão jogando alternadamente e em cada turno, cada um dos jogadores desloca a peça branca para uma das casas livres adjacentes (linhas, colunas e diagonais). As casas livres são as casas ainda não visitadas pela peça branca. Todas os quadrados já visitados são marcados com uma peça preta. Deste modo, depois do movimento da peça branca, coloca-se uma peça negra no quadrado onde a peça branca se encontrava antes da jogada, marcando esse quadrado como visitado.


### Exemplo de uma situação de jogo
Consideremos a situação seguinte, em que é o jogador sul a fazer o próximo lance.


<img src=".\imagens_rastros\tab2.png" alt="Drawing" style="width: 250px;"/>

Aparentemente, parece que a sua jogada natural seria 7-2, ou seja, aproximar-se do seu objectivo. Mas, perante essa jogada, o segundo jogador deslocará a branca para 6-3, afastando o primeiro jogador do seu quadrado final.

<img src=".\imagens_rastros\tab3.png" alt="Drawing" style="width: 250px;"/>

O mesmo acontecerá se escolher 7-1. Nesse caso, o adversário (jogador norte) jogará para 6-1, afastando-o do seu objectivo. 

<img src=".\imagens_rastros\tab4.png" alt="Drawing" style="width: 250px;"/>

A coisa ficaria ainda pior se sul agora decidisse atacar, aproximando-se do objectivo, jogando para 7-2. Nesse caso, o segundo jogador defende-se para 6-3 e ficaria apenas um corredor livre para o quadrado final s, o que não é nada favorável ao jogador sul.


<img src=".\imagens_rastros\tab5.png" alt="Drawing" style="width: 250px;"/>

É importante para um jogador não fechar os caminhos para o respectivo quadrado objectivo. Notem que um jogador com os caminhos para o respectivo objectivo totalmente fechados não está ainda derrotado, ele pode ainda ganhar, bloqueando o adversário. No entanto, parece ser um sinal de bom senso manter um caminho aberto para o objectivo. Assim, parece que é melhor jogar para trás, para a casa 5-3.


<img src=".\imagens_rastros\tab6.png" alt="Drawing" style="width: 250px;"/>

### Representação dos estados do jogo

Para construir as funções de avaliação é necessário perceber como são representadas as diferentes situações do tabuleiro no decurso dos jogos.

Um estado do jogo é um triplo:

```python 
EstadoRastros(Quem_Joga, Posição_Branca, Conjunto_Posições_Pretas)
```

* Quem_Joga corresponde ao do jogador (norte ou sul) que vai jogar. 
* Posição_Branca refere-se à casa onde está colocada a peça branca
* Conjunto_Posições_Pretas é o conjunto com as casas que já foram ocupadas. 

Cada casa do tabuleiro é representada por um par de coordenadas (coluna, linha). 





### Estado inicial
O estado que representa a situação inicial é um triplo: 

```python
EstadoRastros(to_move='S', white=(4, 5), blacks={})
```

Se o jogador Sul abrir, jogando para a casa (4, 4), o tabuleiro passa a ser representado por

```python
EstadoRastros(to_move='N', white=(4, 4), blacks={(4, 5)})
```

E se o jogador Norte responder, jogando para a casa (3, 3) ficaremos com 

```python
EstadoRastros(to_move='S', white=(3, 3), blacks={(4, 5), (4, 4)})
```

e assim sucessivamente.


### Classe EstadoRastros

Veja a definição da classe ```EstadoRastros```, como está no ficheiro ```rastros.py```.

```python
from jogos import *

stateRastros = namedtuple('EstadoRastros', 'to_move, white, blacks')

class EstadoRastros(stateRastros):

    def ve_se_terminou(self):
        "devolve 1 se ganhou sul, -1 se ganhou norte, 0 se não terminou"
        if self.blacks==set():
            return 0
        justplayed = self.other(self.to_move) 
        if self.white==(8,1):
            return 1 
        elif self.white == (1,8):
            return -1 
        elif len(self.moves()) == 0:
            return 1 if justplayed == 'S' else -1
        else:
            return 0
        
    def __init__(self, to_move, white, blacks):
        self.fullboard = set([(x, y) for x in range(1, 9)
                 for y in range(1, 9)])
        self.terminou = self.ve_se_terminou() # = 1 se ganhou sul, -1 se ganhou norte, 0 se não terminou

    def moves(self):
        "Legal moves are any square adjacent to white if not in blacks"
        alladjacent = [(self.white[0]+a, self.white[1]+b) for a in [-1,0,1] for b in [-1,0,1]]
        return [p for p in alladjacent
                if p not in self.blacks and p !=self.white and p in self.fullboard]

    def compute_utility(self, player):
        "If player wins in this state, return 1; if otherplayer wins return -1; else return 0."
        return self.terminou if player=='S' else -self.terminou

    def other(self,player):
        return 'N' if player == 'S' else 'S'

    def posicao(self, a, b):
        if (a,b)==self.white:
            return 'B' 
        elif (a,b) in self.blacks:
            return 'P'
        else:
            return '.'
        
    def display(self):
        print(" 12345678")
        for x in range(1, 9):
            print(x, end="")
            for y in range(1, 9):
                print(self.posicao(x, y), end='')
            print(x)
        print(" 12345678 ")

estado_inicial = EstadoRastros(to_move = 'S', white = (4,5), blacks=set())

```

### Movimentos das peças e alguns predicados disponíveis e que podem ser úteis.
Além da representação do tabuleiro é conveniente conhecer algumas funções já definidas que poderão ser úteis para a construção da função de avaliação e para a implementação das jogadas.

Na classe Rastros estão definidas as funções obrigatórias de Game:

* ```python actions(state)``` : devolve uma lista de jogadas possíveis a partir de state
* ```python result(state, move)```: devolve o novo estado que resulta de fazer a jogada move no estado state
* ```python utility(state, player)```: devolve 1 para uma vitória de player e -1 para uma derrota de player

Estão ainda definidas as funções
* ```python __init__()```: define os atributos fullboard e initial (este é obrigatório)
* ```python terminal_test(state)```: verifica se no estado state o jogo já terminou
* ```pythondisplay()```: mostra o tabuleiro e indica o próximo jogador a jogar


### Classe Rastros

Já agora, aqui está a classe ```Rastros```, como definida em ```rastros.py```

```python
class Rastros(Game):
    """Play rastros on an 8 x 8 board, with Max (first player) playing 'S'.
    A state has the player to move, a cached utility, a list of moves in
    the form of a list of (x, y) positions, and a board, represented by the
    position of the white mark and a list of positions of the black marks."""

    def __init__(self):
        self.fullboard = set([(x, y) for x in range(1, 9)
                 for y in range(1, 9)])
        self.initial = EstadoRastros(to_move = 'S', white = (4,5), blacks=set())

    def actions(self, state):
        "Legal moves are any square adjacent to white if not in blacks"
        return state.moves()

    def result(self, state, move):
        blacks = state.blacks.copy() # Sim, temos de duplicar o conjunto de blacks
        blacks.add(state.white) ## marca a antiga white como black
        return EstadoRastros(to_move=('N' if state.to_move == 'S' else 'S'),
                         white=move,blacks=blacks) 

    def utility(self, state, player):
        "Return the value to player; 1 for win, -1 for loss, 0 otherwise."
        "If the player is S and .utility == 1 then return .utility"
        "Otherwise return the symmetric. Note that the symmetric of 0 is 0"
        "Note that player might be different from the player within the state that has just virtually played"
        aux = self.compute_utility(state)
        return aux if player == 'S' else -aux

    def terminal_test(self, state):
        "A state is terminal if someone won or there are no empty squares."
        "It assumes that the calculus if there is a winner is computed first and saved in .utility, thus it uses the value of .utility."
        return state.terminou != 0


    def display(self, state):
        print("Tabuleiro:")
        state.display()
        fim = self.terminal_test(state)
        if  fim:
            print("FIM do Jogo")
        else :
            print("Próximo jogador:{}\n".format(state.to_move))

```

In [1]:
from rastros import *

ModuleNotFoundError: No module named 'func_timeout'

## Jogadores 
Como exemplificado no ```Rastros.py```, as instâncias da classe ```Jogador``` incluem um nome e uma função de escolha da jogada a efectuar. Esses jogadores podem fazer uso ou não do algoritmo minimax e a sua variante alfabeta.

#### o Bacoco escolhe uma jogada aleatória


In [ ]:
def bacoco(game, state):
    return random.choice(state.moves())

bacoco = Jogador("Bacoco", bacoco)

#### O obtusoSW escolhe sempre a casa mais a Sul e de entre essas, a mais a Oeste


In [ ]:
def sudoeste(game, state):
    moves = state.moves()
    moves.sort(key = lambda t: (t[0],-t[1]))
    return moves[-1]

obtusoSW = Jogador("ObtusoSW", sudoeste)

#### o ObtusoNE escolhe sempre a casa mais a Norte e de entre essas, a mais a Leste

In [ ]:
def nordeste(game, state):
    moves = state.moves()
    moves.sort(key = lambda t: (-t[0],t[1]))
    return moves[-1]

obtusoNE = Jogador("ObtusoNE", nordeste)

#### Dois humanos para poderem jogar vocês mesmos

In [ ]:
def pergunta(game, state):
    state.display()
    print("Jogadas possíveis: ", state.moves())
    return eval(input(state.to_move+", para onde quer jogar? "))
    
humano1 = Jogador("Pessoa1", pergunta)
humano2 = Jogador("Pessoa2", pergunta)

### Jogar e ver o jogo

No ficheiro ```rastros.py```estão definidas funções:

```jogaRastros11(jog1, jog2)``` que faz um jogo entre ```jog1``` e ```jog2```, sendo ```jog1``` a abrir. Devolve a lista de jogadas feita e o resultado.

```mostraJogo(listajog, verbose = False, step_by_step=False)``` para ver a sequência de jogadas efectuada no tabuleiro. Se invocar ```mostraJogo```com ```verbose = True``` tem mais detalhes, e se invocar com ```step_by_step = True``` tem de dar ```<enter>```para o próximo tabuleiro ser mostrado.

```jogaRastros11com_timeout(jog1, jog2, nsec)``` faz também 1 jogo entre ```jog1``` e ```jog2```, sendo ```jog1``` a abrir, mas em que cada jogada tem o limite de tempo de ```nsec``` segundos. Se um dos jogadores demorar mais do que o tempo limite perde o jogo.

```jogaRastrosNN(listaJog, listaAdv, n, nsec)``` que faz um campeonato (com timeout) entre os jogadores de ```listaJog``` e os jogadores de ```listaAdv```. Todos os pares de jogadores jogam entre si um nó 2n de jogos, n como primeiro jogador (sul) e outros n como segundo (norte).

#### ObtusoSW contra Bacoco , jogamos primeiro e vemos o resultado

In [ ]:
jogo1 = jogaRastros11(obtusoSW, bacoco)
print(jogo1)

#### e depois vemos o jogo 

In [ ]:
mostraJogo(jogo1[0])

#### para ver os tabuleiros

In [ ]:
mostraJogo(jogo1[0], verbose = True)

### Jogadores Alfabeta
que fazem uso da variante limitado do algoritmo minimax-alfabeta, implementado na função ***alphabeta_cutoff_search_new***.

#### O Random Plus
O Random Plus prefere ganhar e detesta perder, fora isso, todas as jogadas valem o mesmo. Ele está limitado a uma profundidade 5.

In [ ]:
depth = 5

def f_aval_randomPlus(estado, jogador):
    if estado.terminou == 1:
        return 10 if jogador == "S" else -10
    elif estado.terminou == -1:
        return 10 if jogador == "N" else -10
    else:
        return 0

randomPlus = Jogador("Random Plus",
                  lambda game, state:
                  alphabeta_cutoff_search_new(state,game,depth,eval_fn=f_aval_randomPlus))

#### Vamos fazer um jogo entre o Bacoco e o Random Plus

In [ ]:
jogo1 = jogaRastros11(bacoco, randomPlus)
mostraJogo(jogo1[0],True)

#### O jogador básico 'Basílio', 
que reconhece estados de vitória e de derrota e em jogos ainda não terminados valoriza apenas a distância da peça Branca à casa objectivo de cada jogador.

In [ ]:
##### heuristica para jogador básico
# derrota vale -10, vitória vale 10,
# cc subtrai distância da peça branca à casa objectivo

depth=5

def distancia (a, b):
    return max(abs(a[0]-b[0]), abs(a[1]-b[1]))

def f_aval_basico(estado, jogador):
    if estado.terminou == 1:
        return 10 if jogador == "S" else -10
    elif estado.terminou == -1:
        return 10 if jogador == "N" else -10
    else:
        obj = (8, 1) if jogador == "S" else (1, 8)
        return 7-distancia(estado.white, obj)

basilio = Jogador("Basilio",
                  lambda game, state:
                  alphabeta_cutoff_search_new(state,game,depth,eval_fn=f_aval_basico))


#### ObtusoSW vs Basílio

In [ ]:
jogo1 = jogaRastros11(obtusoSW, basilio)
print(jogo1)
mostraJogo(jogo1[0],True)

In [ ]:
jogo1 = jogaRastros11(randomPlus, basilio)
mostraJogo(jogo1[0],True)

#### O Ar Livre
O Ar Livre prefere ter mais jogadas possíveis pensando que o seu adversário prefere que tenha menos jogadas, menos ar.

In [ ]:
#### funções heurísticas para avaliação de estado
def num_livres(estado,jogador) :
    """Se ganha 10 e se perde -10 mas se não ganhar nem perder
    maximiza o espaço livre junto à peça B"""
    
    if estado.terminou == 1:
        return 10 if jogador == "S" else -10
    elif estado.terminou == -1:
        return 10 if jogador == "N" else -10
    else:
        return len(estado.moves())
   

#### mais um jogador, agora com alpha-beta
arlivre = Jogador("Ar Livre",
                  lambda game, state:
                  alphabeta_cutoff_search_new(state,game,depth_for_all,eval_fn=num_livres))


#### Ar Livre contra Basílio
Notem que ambos os jogadores fazem uso do alfabeta à mesma profundidade, 5

<img src=".\imagens_rastros\duel.gif" alt="Drawing" style="width: 200px;"/>

In [ ]:
jogo1 = jogaRastros11(arlivre, basilio)
mostraJogo(jogo1[0],True)

In [ ]:
jogo1 = jogaRastros11(basilio,arlivre)
mostraJogo(jogo1[0],True)

#### Campeonato
Podemos fazer um campeonato todos contra todos , neste caso com todos os jogadores definidos em rastros.py (excepto os humanos), em que cada par de jogadores realiza um nº 2xn de jogos, n como Sul e os outros n como Norte

In [ ]:
todosJog = [bacoco, randomPlus,obtusoSW, obtusoNE, arlivre, basilio]
campeonato = jogaRastrosNN(todosJog, todosJog, n=5,nsec=10)

<img src=".\imagens_rastros\mac-thinking.gif" alt="Drawing" style="width: 250px;"/>

#### Agora construímos a tabela classificativa
Para isso precisamos de executar este código:

In [ ]:
resultado_jogos = [(a,b,n) for (a,b,(x,n)) in campeonato]
tabela = dict([(jog.nome, 0) for jog in todosJog])
for jogo in resultado_jogos:
    if jogo[2] == 1:
        tabela[jogo[0]] += 1
    else:
        tabela[jogo[1]] += 1
classificacao = list(tabela.items())
classificacao.sort(key=lambda p: -p[1])
print("JOGADOR", "VITÓRIAS")
for jog in classificacao:
    print('{:11}'.format(jog[0]), '{:>4}'.format(jog[1]))

#### A função ```faz_campeonato(listaJogadores, n, nsec)```faz isto mesmo, basta invocar com todosJog, ou com a lista de jogadores que se quiser, por exemplo

In [ ]:
faz_campeonato([bacoco, arlivre, randomPlus,basilio,obtusoSW, obtusoNE],10)

<img src=".\imagens_rastros\duelo_esgrima.gif" alt="Drawing" style="width: 200px;"/>